### Imports

In [28]:
from collections import namedtuple
from queue import PriorityQueue
from functools import reduce
from random import random

import numpy as np
import math

### Constants

In [29]:
PROBLEM_SIZE = 500
NUM_SETS = 1000
PROBABILITY = 0.3

SETS = tuple(
    np.array([random() < PROBABILITY for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)

State = namedtuple('State', ['taken', 'not_taken'])

### Functions

In [30]:
# Goal Check Function

def goal_check(state):
    return np.all(reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ))

assert goal_check(
    State(set(range(NUM_SETS)), set())
), "Problem not solvable"

In [31]:
# Cost Function

def g(state):
  return len(state.taken)

In [32]:
# Heuristic Function

def h(state):
  return PROBLEM_SIZE - sum(
    reduce(
      np.logical_or,
      [SETS[i] for i in state.taken],
      np.array([False for _ in range(PROBLEM_SIZE)]),
    )
  )

In [34]:
def h2(state):
  uncovered = np.array([False for _ in range(PROBLEM_SIZE)])

  for i in state.taken:
    uncovered |= SETS[i]

  num_sets_needed = sum(~uncovered)
  return num_sets_needed

In [38]:
def h3(state):
    uncovered = np.array([False for _ in range(PROBLEM_SIZE)])

    for i in state.taken:
        uncovered |= SETS[i]

    best_set_to_add = -1
    max_coverage = -1

    for i in state.not_taken:
        # Calculate the number of uncovered elements this set would cover
        set_coverage = sum(SETS[i] & ~uncovered)

        if set_coverage > max_coverage:
            max_coverage = set_coverage
            best_set_to_add = i

    return max_coverage

In [39]:
# Sum Function

def f(state):
  return g(state) + h3(state)

### Execution

In [ ]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    print(counter)
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put((f(new_state), new_state))
    _, current_state = frontier.get()

### Info

In [41]:
print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

Solved in 9 steps (9 tiles)
